# Mental Health in Tech Industry Analysis
### *author: Celine Ng*
### *April 2024*

# Table of Contents 
1. Notebook Preparation
    1.1 Libraries Installation
    1.2 Data Retrieval
    1.3 Initial Data Cleaning
2. 

# 1. Notebook preparation
## 1.1 Libraries Installation

In [1]:
!pip install pandas
!pip install matplotlib
!pip install seaborn

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql

## 1.2 Data Retrieval

In [3]:
address = 'mental_health.sqlite'
